# 実験3-5 自己教師あり学習

自己教師あり学習は，比較的最近に提案された学習手法で，ラベルを必要としない教師なし学習の手法として着目されています．
その基本的な考え方は，

- 同じデータから派生されるデータ拡張は同じ（似たような）表現にマップされるべき
- 異なるデータ（のデータ拡張）は，異なる表現になるべき

というアイディアに基づいた学習手法です．
教師あり学習では，ラベルとよばれる絶対的な指標がありますが，ここでは，表現が似るべきというわりと曖昧なコンセプトに基づいて学習を考えていきます．

ここでは SimCLR （の簡易版）を考えてみます．
キーポイントはデータ拡張の部分にあります．

ここではランダムな，画像切り抜き，フリップ，色補正などを考えています．
データセットとしては `CIFAR10` を考えます．

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.ToTensor()
])

dataset = datasets.CIFAR10(root='./data', download=True, transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

ネットワークの構成としては，表現を得るためのモデル(`encoder`)と，その表現を投影し似ているかどうかを判定するためのモデル(`projector`)を規定します．

In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.projector = nn.Sequential(
            nn.Linear(8*8*128, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.projector(x)
        return x

損失関数は，投影された先の特徴量間の類似度を最大化するようにコサイン類似度で設計します．

In [25]:
criterion = nn.CosineEmbeddingLoss()

さらに学習ループは，下記のようにかきます．

In [ ]:
from torchvision.transforms import ToPILImage

# ToPILImageインスタンスを作成
to_pil = ToPILImage()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleCNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    for (images, _) in dataloader:
        # 画像ペア生成
        
        # 画像を PIL に変換してデータ拡張を2回適用
        images1 = torch.stack([transform(to_pil(img)) for img in images])
        images2 = torch.stack([transform(to_pil(img)) for img in images])

        images1, images2 = images1.to(device), images2.to(device)

        # 特徴抽出
        z1 = model(images1)
        z2 = model(images2)

        # 類似性損失の計算
        targets = torch.ones(z1.size(0)).to(z1.device)  # 正例ペア
        loss = criterion(z1, z2, targets)

        # 学習
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


In [ ]:
# 特徴を抽出して固定
for param in model.parameters():
    param.requires_grad = False

# 簡単な分類器を訓練
classifier = nn.Linear(128, 10)
optimizer = optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# CIFAR-10のラベルを使って学習
for epoch in range(5):
    for (images, labels) in dataloader:
        features = model(images)
        outputs = classifier(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# 実験3-5

1. 自己教師あり学習による識別器を構成し，`CIFAR10` を用いた場合の識別性能を評価しなさい．（識別器はロジスティック回帰やSVM を用いて構わない）
2. 自己教師あり学習によって得られた `CIFAR10` の特徴表現を，PCA や t-SNE を用いて図示し，各クラスのデータが構造を持つかどうかを考察しなさい．
3. 自己教師あり学習によって得られた `CIFAR10` の特徴表現を k-means 法によりクラスタリングを行い，自己教師あり学習の特徴がクラスタリングに有効かどうかを評価しなさい．